Global setup

In [1]:
n=4
k=2
q=41 #Alkaline D
#q should be divisible by 8 for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
R.<x> = ZZ[]

In [2]:
q

41

In [3]:
is_prime(q)

True

In [4]:
for tau in range(5):
    print(ln(binomial(4, tau)).n()+tau) #challenge entropy, should be between n/2 and n

0.000000000000000
2.38629436111989
3.79175946922805
4.38629436111989
4.00000000000000


In [5]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [6]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [7]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.586530272919658

Expected number of repetitions:

In [8]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))^2.n()

2.90682666192675

Alice generates a signing key

In [9]:
A = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
A

[                 6*x^2 + 36  2*x^3 + 14*x^2 + 22*x + 14]
[    40*x^3 + x^2 + 28*x + 9 19*x^3 + 29*x^2 + 36*x + 15]

In [10]:
s1 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s1

[        -x^3]
[-x^2 - x - 1]

In [11]:
s2 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s2

[-x^3 - x^2]
[  -x^3 - 1]

$s_1$ and $s_2$ are private

In [12]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[  7*x^3 + 31*x^2 - 28*x + 2]
[29*x^3 - 40*x^2 - 31*x + 19]

$A$ and $t$ are public

Alice encodes her message

In [42]:
M = 'HI_BOB__'
Mlist = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
Mlist

[8, 9, -10, 2, 15, 2, -10, -10]

In [35]:
y1 = matrix([[randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)], 
            [randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)]])
y1

[  6*x^3 - 12*x^2 + 3*x + 14]
[-15*x^3 - 10*x^2 - 11*x - 8]

In [36]:
y2 = matrix([[randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)], 
            [randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)]])
y2

[-15*x^3 - 7*x^2 + 12*x - 11]
[   -3*x^3 - 2*x^2 - 9*x - 8]

In [37]:
w1 = matrix(((A*y1+y2) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w1

[-12*x^3 + x^2 - 16*x + 2]
[ x^3 - 7*x^2 - 10*x - 18]

Start the "non-cryptographic hash"

In [41]:
w1list = [c for row in w1 for p in row for c in list(p)]  #flatten the matrix down to coefficients
w1list

[2, -16, 1, -12, -18, -10, -7, 1]

In [45]:
(vector(w1list)-vector(Mlist)) % 64

(58, 39, 11, 50, 31, 52, 3, 11)

In [55]:
[format(c, '06b') for c in (((vector(w1list)-vector(Mlist)) % 64))]

['111010',
 '100111',
 '001011',
 '110010',
 '011111',
 '110100',
 '000011',
 '001011']

In [54]:
"".join([format(c, '06b') for c in (((vector(w1list)-vector(Mlist)) % 64))])

'111010100111001011110010011111110100000011001011'

In [20]:
def sample_in_ball(digest):  # digest should be twice the length of c
    c = [0] * 4
    for i in range(4-tau, 4):
        j = digest[i] % (i+1)
        s = digest[i+4] % 2
        c[i] = c[j]
        c[j] = (-1)^s
    return c
    

In [21]:
c = R(sample_in_ball(c_hash.digest(8)))
c

-x^2

In [22]:
z1 = matrix((y1 + c*s1) % (x^n+1))
z1

[-13*x^3 - 10*x^2 - 12*x + 3]
[   6*x^3 + 3*x^2 + 13*x + 7]

In [23]:
z2 = matrix((y2 + c*s2) % (x^n+1))
z2

[     -15*x^3 - 2*x - 3]
[7*x^3 + x^2 + 6*x + 12]

Is $\|\mathbf{z}_1\|_\infty$ or $\|\mathbf{z}_2
\|_\infty \geq \gamma_1-\beta$?

In [24]:
gamma1-beta

15

If so, reject and pick new $\mathbf{y}_i$

If not, $(\mathbf{z}_1, \mathbf{z}_2, c)$ is the signature

In [25]:
(z1, z2, c)

(
[-13*x^3 - 10*x^2 - 12*x + 3]  [     -15*x^3 - 2*x - 3]      
[   6*x^3 + 3*x^2 + 13*x + 7], [7*x^3 + x^2 + 6*x + 12], -x^2
)

Bob verifies the signature

In [26]:
w1prime = matrix((A*z1+z2-c*t) % (x^n+1) % q).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w1prime

[   9*x^3 - 8*x^2 + 6*x - 9]
[-15*x^3 - 19*x^2 + 8*x - 4]

Is $\|\mathbf{z}_i\|_\infty < \gamma_1-\beta$?

In [27]:
z1

[-13*x^3 - 10*x^2 - 12*x + 3]
[   6*x^3 + 3*x^2 + 13*x + 7]

In [28]:
z2

[     -15*x^3 - 2*x - 3]
[7*x^3 + x^2 + 6*x + 12]

In [29]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [30]:
cprime_hash = hashlib.shake_256(str_to_bytes(M+str(w1prime)))

In [31]:
cprime = R(sample_in_ball(cprime_hash.digest(8)))
cprime

-x^2

In [32]:
c == cprime

True

Yes.  The signature is accepted.